In [1]:
import sys
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")

from MARScore.utils import * 
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import hdbscan
from sklearn.metrics import make_scorer
from random import seed
from datasets_loaders.loaders import load_billsum

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotat

In [2]:
billsum = load_billsum()
subset = billsum.iloc[:200, :]

In [3]:
def DBCV(model, X, y=None):
    preds = model.fit_predict(X)
    return hdbscan.validity.validity_index(X, preds) if len(set(preds)) > 1 else float('nan')

def HDBScanFinetune(vectors, 
                    min_samples=[10,30,50,60,100], 
                    min_cluster_size=[100,200,300,400,500,600],
                    cluster_selection_method=['eom','leaf'],
                    seed_num=0, 
                    verbose=True):
    
    #model setup
    hdb = hdbscan.HDBSCAN(gen_min_span_tree=True)

    # specify parameters and distributions to sample from
    param_dist = {'min_samples': min_samples,
                  'min_cluster_size': min_cluster_size,  
                  'cluster_selection_method': cluster_selection_method
                 }

    #validity_scroer = "hdbscan__hdbscan___HDBSCAN__validity_index"
    #validity_scorer = make_scorer(DBCV, greater_is_better=True)

    #parameters research
    """
    random_search = GridSearchCV(hdb,
                                 param_grid=param_dist,
                                 scoring=validity_scorer)
    """
    
    n_iter_search = 2
    random_search = RandomizedSearchCV(hdb,
                                       param_distributions=param_dist,
                                       n_iter=n_iter_search,
                                       scoring=DBCV,
                                       random_state=seed(seed_num))
    
    random_search.fit(vectors)

    if verbose:
        print(f"Best Parameters {random_search.best_params_}")
        print(f"DBCV score :{random_search.best_estimator_.relative_validity_}")
    return {"best_params": random_search.best_params_, "dbcv_score":random_search.best_estimator_.relative_validity_}

In [ ]:
"""
def to_shape(a, shape):
    x_, y_, z_ = shape
    x, y, z = len(a), len(a[0]), len(a[0][0])
    x_pad = (x_-x)
    y_pad = (y_-y)
    z_pad = (z_-z)
    return np.pad(a,((0, x_pad),
                     (0, y_pad),
                     (0, z_pad)),
                  mode = 'constant')
"""

In [4]:
def to_shape(a, shape):
    x_, y_ = shape
    x, y = len(a), len(a[0])
    x_pad = (x_-x)
    y_pad = (y_-y)
    return np.pad(a,((0, x_pad),
                     (0, y_pad)),
                  mode = 'constant')

In [5]:
#creation of embeddings
all_v = []
for indiv in subset["text"].to_list():
    o, l = tokenizeCorpus(indiv)
    v = vectorizeCorpus(o)
    v, l = cleanAll(v, l)
    all_v.append(v)

In [ ]:
max_dim_1 = np.max([len(x) for x in all_v])
all_v2 = np.array([to_shape(cur_v, (max_dim_1, len(all_v[0][0]))) for cur_v in all_v])
s = all_v2.shape
all_v2 = all_v2.reshape((s[0], s[1]*s[2]))

In [ ]:
np.array(all_v[0]).shape

In [ ]:
all_v2.shape

In [ ]:
HDBScanFinetune(all_v2,
                min_samples=[1, 3, 5, 10],
                min_cluster_size=[1, 5, 10, 20],
                cluster_selection_method=['eom','leaf'])

In [6]:
max_dim_1 = np.max([len(x) for x in all_v])
all_v3 = [to_shape(cur_v, (max_dim_1, len(all_v[0][0]))) for cur_v in all_v]

In [7]:
DBCV(hdbscan.HDBSCAN(), all_v3[0])

0.6898272642990181

In [10]:
HDBScanFinetune(all_v3[0],
                min_samples=[1, 3, 5, 7],
                min_cluster_size=[7, 10, 20],
                cluster_selection_method=['eom','leaf'])

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan]
  warnings.warn(


Best Parameters {'min_samples': 3, 'min_cluster_size': 7, 'cluster_selection_method': 'leaf'}
DBCV score :0.67558658664357


{'best_params': {'min_samples': 3,
  'min_cluster_size': 7,
  'cluster_selection_method': 'leaf'},
 'dbcv_score': 0.67558658664357}